In [1]:
import pandas as pd
from IPython.display import display
import folium
from folium.features import CustomIcon
import re
import requests
pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv('../data/oakland_fruits_with_coords.csv')

In [3]:
# available fruits:
# 'plum',       'loquat',    'lemon', 
# 'pear',       
# 'crabapple',  'apple',     'persimmon',   'avocado',
# 'guava',
wanted_trees = ['plum']

tree_name_to_species_match = {
#     plum
    "plum": [
            "Prunus domestica",
#             "Prunus sp",
             "Prunus cerasifera",
#             "Prunus cerasifera/blireiana"
            ],
#     loquat
    "loquat": ["Eriobotrya japonica"], 
#     lemon
    "lemon": 
        ["Citrus sp"],
    "crabapple": 
        ["Malus ioensis", "Malus floribunda"],
#     apple
    "apple": 
        ["Malus sp"],
#     Persimmon
    "persimmon": ["Diospyros kaki"],
#     Avocado
    "avocado": ["Persea sp / avocado"],
#     guava
    "guava": ["Feijoa sellowiana"],
    
    }

fruit_trees = []

# case where no specific trees were requested
if len(wanted_trees) == 0:
    fruit_trees = [fruit for sublist in tree_name_to_species_match.values() for fruit in sublist]        
# case where we DO NOT want certain trees
elif wanted_trees[0][0] == "!":
    unwanted_trees = [tree[1:] for tree in wanted_trees]
    for common_name in tree_name_to_species_match.keys():
        if common_name in unwanted_trees:
            continue
        fruit_trees += tree_name_to_species_match[common_name]
# case where we only want certain fruits
else:
    for fruit_name in wanted_trees:
        fruit_trees += tree_name_to_species_match[fruit_name]

fruit_trees, len(fruit_trees)

(['Prunus domestica', 'Prunus cerasifera'], 2)

In [4]:
fruits = df.dropna(subset=['SPECIES'])
# Convert the fruit names to lowercase
fruit_trees_lower = [fruit.lower() for fruit in fruit_trees]

# Filter the DataFrame based on exact matches
fruits = fruits[fruits['SPECIES'].str.lower().isin(fruit_trees_lower)]
# fruits = fruits[fruits['SPECIES'].str.contains('|'.join(fruit_trees), case=False)]

fruits

,OBJECTID,WELLWIDTH,WELLLENGTH,PAREAWIDTH,LOWWELL,SPECIES,STNAME,Location 1,StreetNumber,Address,Latitude,Longitude
83,9593,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
131,14510,2.0,2.0,0.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
172,19292,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
173,19294,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
174,19305,0.0,0.0,6.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
175,19307,2.0,2.0,0.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
355,38580,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
356,38587,0.0,0.0,6.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
357,38583,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029
358,38579,0.0,0.0,5.0,NaN,Prunus cerasifera,NaN,"0\nOakland, Ca\n(37.80505999961946, -122.27301...",0,NaN,46.314475,11.048029


In [5]:
# tree specifics
tree_info = {
#     Cherry
    "Prunus avium": {
        "popup_title": "Cherry tree",
        "image_path": '../resources/cherry.png',
        "icon_size": (25, 25),
    },
#     Plum
    "Prunus domestica": {
        "popup_title": "Plum tree",
        "image_path": '../resources/plum.png',
        "icon_size": (20, 20),
    },
#     Plum
        "Prunus cerasifera": {
        "popup_title": "Plum tree",
        "image_path": '../resources/plum.png',
        "icon_size": (20, 20),
    },
#     Loquat
    "Eriobotrya japonica": {
        "popup_title": "Loquat tree",
        "image_path": '../resources/loquat.png',
        "icon_size": (20, 20),
    },
#   Lemon
    "Citrus sp": {
        "popup_title": "Lemon tree",
        "image_path": '../resources/lemon.png',
        "icon_size": (25, 25),
    },
#     Crabapple
    "Malus floribunda": {
        "popup_title": "Crabapple tree",
        "image_path": '../resources/crabapple.png',
        "icon_size": (25, 25),
    },
#     Crabapple
    "Malus ioensis": {
        "popup_title": "Crabapple tree",
        "image_path": '../resources/crabapple.png',
        "icon_size": (25, 25),
    },
#     Apple
    "Malus sp": {
        "popup_title": "Apple tree",
        "image_path": '../resources/apple.png',
        "icon_size": (25, 25),
    },
#     Persimmon
    "Diospyros kaki": {
        "popup_title": "Persimmon tree",
        "image_path": '../resources/persimmon.png',
        "icon_size": (25, 25),
    },
#     Avocado
    "Persea sp / avocado": {
        "popup_title": "Avocado tree",
        "image_path": '../resources/avocado.png',
        "icon_size": (20, 20),
    },
#     Guava
    "Feijoa sellowiana": {
        "popup_title": "Guava tree",
        "image_path": '../resources/guava.png',
        "icon_size": (25, 25),
    },
}



In [12]:
# Create a map centered on San Francisco
latitude_center = 37.8735
longitude_center = -122.2717
map_center = [latitude_center, longitude_center]
map_zoom = 12.5  # Adjust the zoom level as needed
map_obj = folium.Map(location=map_center, zoom_start=map_zoom)

# Add markers with custom icons for each point
for index, row in fruits.iterrows():
    location = row['Location 1']
    street_num = location.split("\n")[0]
    street_name = row["STNAME"]
    address = row["Address"]
    # Extract the latitude and longitude
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    if pd.isna(latitude):
        continue
    
    image_path = tree_info[row['SPECIES']]['image_path']
    tree_type = tree_info[row['SPECIES']]['popup_title']
    icon_size = tree_info[row['SPECIES']]['icon_size']
    maps_link = f"http://www.google.com/maps/place/{latitude},{longitude}"
#     print('hi')
    # Define the path to the image file
    icon = CustomIcon(
            icon_image=image_path,
            icon_size=icon_size,  # Adjust the size as needed
#             icon_anchor=(25, 25),  # Adjust the anchor position as needed
        )

    marker = folium.Marker(
        location=[latitude, longitude], 
        icon=icon,
        tooltip=tree_type
    )
    title = tree_type + ": " + row["SPECIES"]
    popup_content = f"<strong>{title}</strong><br><strong>Address:</strong> {address}<br><strong>Google Maps URL:</strong> <a href='{maps_link}' target='_blank'>{maps_link}</a>"
    popup = folium.Popup(popup_content)  # Adjust max_width as needed
    marker.add_child(popup)
    
    marker.add_to(map_obj)

# Display the map


In [13]:
map_obj

In [ ]:
address = "5825 COLBY ST Oakland, CA"
response = requests.get(f"https://nominatim.openstreetmap.org/search?q={address}&format=json")


In [ ]:
response.text


In [ ]:
dir(response)

In [ ]:
def add_address(fruits):
    fruits['StreetNumber'] = fruits['Location 1'].str.split('\n').str[0]
    # Create the 'Address' column by concatenating street number, street name, and city/state
    fruits['Address'] = fruits['StreetNumber'] + ' ' + fruits['STNAME'] + ', Oakland, CA'
    return fruits

fruits = add_address(fruits)

def get_coordinates(address):
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={address}&format=json")
    data = response.json()
    if data:
        latitude = float(data[0]['lat'])
        longitude = float(data[0]['lon'])
        return latitude, longitude
    else:
        print('something went wrong')
        return None, None
    
def add_coords(fruits):
    fruits[['Latitude', 'Longitude']] = fruits['Address'].apply(get_coordinates).apply(pd.Series)
    return fruits

fruits = add_coords(fruits)


In [ ]:
addresses = list(fruits["Address"])
lats = []
longs = []
for address in addresses:
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={address}&format=json")
    data = response.json()
    if data:
        lats += [float(data[0]['lat'])]
        longs += [float(data[0]['lon'])]
        print(float(data[0]['lat']), float(data[0]['lon']))
        
    else:
        print('something went wrong')
        lats += [""]
        longs += [""]


In [ ]:
lats

In [ ]:
messed_up_ind = []
for i in range(len(lats)):
    if lats[i] == '':
        messed_up_ind.append(i)
messed_up_ind

In [ ]:
messed_up_addresses = []
addresses = list(fruits["Address"])

for i in messed_up_ind:
    messed_up_addresses.append(addresses[i])
messed_up_addresses

In [ ]:
messed_up_addresses = ['3027 RICHMOND, Oakland, CA',
 '4163 LAKE SHORE, Oakland, CA',
 '5936 Chabolyn Terrace, Oakland, CA',
 '2600 HAVENSCOURT, Oakland, CA',
 '636 HILLGIRT, Oakland, CA',
 '776 MANDANA, Oakland, CA',
 '0 SKYLINE, Oakland, CA',
 '5500 Martin Luther King Jr Way, Oakland, CA',
 '5820 Martin Luther King Jr Way, Oakland, CA',
 '636 HILLGIRT, Oakland, CA',
 '3251 Martin Luther King Jr Way, Oakland, CA',
 '3251 Martin Luther King Jr Way, Oakland, CA',
 '0 MARSHALL, Oakland, CA',
 '4257 LAKE SHORE AVE, Oakland, CA']

In [ ]:
addresses = messed_up_addresses
lats = []
longs = []
for address in addresses:
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={address}&format=json")
    data = response.json()
    if data:
        lats += [float(data[0]['lat'])]
        longs += [float(data[0]['lon'])]
        print(float(data[0]['lat']), float(data[0]['lon']))
        
    else:
        print('something went wrong')
        lats += [""]
        longs += [""]

In [ ]:
address_list = fruits['Address'].tolist()

# Replace the addresses at the specified indices in the list
for i, idx in enumerate(messed_up_ind):
    address_list[idx] = messed_up_addresses[i]
    
fruits['Address'] = address_list


In [ ]:
fruits = add_coords(fruits)
fruits

In [ ]:
fruits.to_csv('oakland_fruits_with_coords.csv', index=False)